In [80]:
import pandas as pd
import numpy as np

df = pd.read_csv('poetry.csv')
df = df.drop(columns=df.columns[[5, 6, 7, 8]], axis=1)

df = df[:]

def get_century_roman(year):
	century = year // 100 + 1
	roman_numerals = {
		1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V',
		6: 'VI', 7: 'VII', 8: 'VIII', 9: 'IX', 10: 'X',
		11: 'XI', 12: 'XII', 13: 'XIII', 14: 'XIV', 15: 'XV',
		16: 'XVI', 17: 'XVII', 18: 'XVIII', 19: 'XIX', 20: 'XX', 21: 'XXI'
	}
	return roman_numerals[century]

def proc_row(row):
	name = row['name'] if not pd.isna(row['name']) else row['text'].split('\n')[0]
	author = row['author'] if not pd.isna(row['author']) else 'неизвестен'

	date_from, date_to = row['date_from'], row['date_to']
	if pd.isna(date_from) and not pd.isna(date_to): date_from = date_to
	if pd.isna(date_to) and not pd.isna(date_from): date_to = date_from
	
	century = ''
	epoch = ''
	if pd.isna(date_to) and pd.isna(date_from): 
		date_to = date_from = -1
	else:
		century = f'{get_century_roman(date_to)} ({int(date_to//100+1)})'
		if date_to < 1500: epoch = 'древнерусская поэзия'
		if 1750 <= date_to < 1850: epoch = 'золотой век'
		if 1850 <= date_to < 1900: epoch = 'реализм и натурализм'
		if 1900 <= date_to < 1950: epoch = 'серебряный век, авангард'
		#if 1950 <= date_to < 1985: epoch = 'Советская поэзия'
		if 1990 <= date_to < 2010: epoch = 'бронзовый век'
		if 2010 <= date_to < 2100: epoch = 'современная поэзия'
	
	#text = '\n'.join(row['text'].split('\n')[:4])
	text = row['text'].replace('\r', '')

	line_count = text.count('\n')
	word_count = text.count(' ') + line_count

	return pd.Series({'author': author, 'name': name, 'text': text, 
				   	  'date_from': int(date_from), 'date_to': int(date_to), 'century': century, 'epoch': epoch, 
				  	  'line_count': line_count, 'word_count': word_count})

df = df.apply(proc_row, axis=1)

df.head()

,author,name,text,date_from,date_to,century,epoch,line_count,word_count
0,Михаил Лермонтов,Забывши волнения жизни мятежной...,"Забывши волнения жизни мятежной,\nОдин жил в п...",1829,1829,XIX (19),золотой век,7,29
1,Сергей Есенин,"Нивы сжаты, рощи голы...","Нивы сжаты, рощи голы,\nОт воды туман и сырост...",1917,1918,XX (20),"серебряный век, авангард",11,46
2,Игорь Северянин,ЧАРЫ ЛЮЧИНЬ,Лючинь печальная читала вечером ручьисто-вкрад...,1919,1919,XX (20),"серебряный век, авангард",11,85
3,Анатолий Жигулин,Золото,"Глыбу кварца разбили молотом,\nИ, веселым огне...",1963,1963,XX (20),,29,122
4,Николай Тихонов,Хоровод в Сульдуси,"Хлынул дождь, когда девушки, встав в хоровод,\...",1937,1940,XX (20),"серебряный век, авангард",15,92


In [81]:
len(df)

16694

In [82]:
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
tqdm.pandas()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

emb_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
#emb_model = SentenceTransformer('intfloat/multilingual-e5-large')
emb_model.to(device)

def emb_row(row):
	metadata = f'название: {row["name"]}, '
	if row['author'] != 'неизвестен': metadata += f'автор: {row["author"]}, '

	if row['date_from'] != -1:
		metadata += f'период написания: {row["century"]} век, {row["date_from"]} - {row["date_to"]} год, '

	if len(row['epoch']) > 0: metadata += row['epoch'] + ', '

	metadata += f'объем: количество строк {row["line_count"]}, '
	metadata += f'количество четверостиший: {int(row["line_count"]//4)}, '
	metadata += f'количество слов: {row["word_count"]}'
	
	row['text_embedding'] = emb_model.encode(row['text'])
	row['metadata_embedding'] = emb_model.encode(metadata)

	return pd.Series(row)


df = df.progress_apply(emb_row, axis=1)


Using device: cpu


100%|██████████| 16694/16694 [1:11:16<00:00,  3.90it/s]


In [83]:
df.to_pickle('poetry_data_prepared.pkl')

In [115]:
import pandas as pd

df = pd.read_pickle('poetry_data_prepared.pkl')
df.head()

,Unnamed: 0,author,name,text,date_from,date_to,century,epoch,line_count,word_count,text_embedding,metadata_embedding
0,0,Михаил Лермонтов,Забывши волнения жизни мятежной...,"Забывши волнения жизни мятежной,\nОдин жил в п...",1829,1829,XIX (19),золотой век,7,29,[-9.15968884e-03 2.05528717e-02 -2.54428294e-...,[ 6.93919510e-02 5.78579446e-03 -5.24416007e-...
1,1,Сергей Есенин,"Нивы сжаты, рощи голы...","Нивы сжаты, рощи голы,\nОт воды туман и сырост...",1917,1918,XX (20),"серебряный век, авангард",11,46,[-0.01985577 0.00113188 0.02673516 -0.001073...,[ 2.75703203e-02 3.91281471e-02 -1.82045661e-...
2,2,Игорь Северянин,ЧАРЫ ЛЮЧИНЬ,Лючинь печальная читала вечером ручьисто-вкрад...,1919,1919,XX (20),"серебряный век, авангард",11,85,[-5.21772653e-02 -6.42890483e-03 2.41918731e-...,[ 2.70281769e-02 4.64453958e-02 -4.95559396e-...
3,3,Анатолий Жигулин,Золото,"Глыбу кварца разбили молотом,\nИ, веселым огне...",1963,1963,XX (20),NaN,29,122,[-2.90191974e-02 -1.09357918e-02 3.89384455e-...,[-1.32629194e-03 2.49236841e-02 1.14643015e-...
4,4,Николай Тихонов,Хоровод в Сульдуси,"Хлынул дождь, когда девушки, встав в хоровод,\...",1937,1940,XX (20),"серебряный век, авангард",15,92,[ 2.60227248e-02 2.13094316e-02 -2.77459323e-...,[-2.49936618e-02 5.71375005e-02 -3.75276245e-...


In [114]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


def cosine_similarity(a, b):
	return np.dot(a, b) / np.linalg.norm(a) / np.linalg.norm(b)

def find_poem(request, top_n=10, lim=0.):
	req_emb = emb_model.encode(request)

	texts_datas = []

	for index, row in tqdm(df.iterrows()):
		text_sim = cosine_similarity(req_emb, row['text_embedding'])
		req_sim = cosine_similarity(req_emb, row['metadata_embedding'])
		score = text_sim * req_sim

		if score >= lim:
			texts_datas.append({'poem': row['text'], 'metadata': {k: v for k, v in row.items() if k != 'text' and k != 'metadata_embedding' and k != 'text_embedding'}, 
					   			'score': score, 'text_sim': text_sim, 'req_sim': req_sim})
	
	texts_datas.sort(key=lambda x: x['score'])

	plt.plot([x['text_sim'] for x in sorted(texts_datas, key=lambda x: x['score'])], label='text')
	plt.plot([x['req_sim'] for x in sorted(texts_datas, key=lambda x: x['score'])], label='req')
	plt.show()
	print(np.mean([x['req_sim'] for x in texts_datas]))
	print(np.mean([x['text_sim'] for x in texts_datas]))

	return texts_datas[-top_n:]


#request = 'из окны видны грустные пожелтевшие деревья но все же это моя любимая романтика'
#request = 'научный энтузиазм и борение за оригинальную идею'
#request = 'поднять! сделать! решительность! я! восклецания, Владимир Маяковский'
#request = 'дыр бул щыл убеш щур скум вы со бу р л эз'
request = 'стихотворение, поэзия, поэт, творчество, искусство'
#request = 'стихотворение, поэзия, поэт, творчество, искусство, серебрянный век, Автор: Владимир Маяковский'
#request = 'Автор: Александр Крученых Автор: Александр Крученых Автор: Александр Крученых Автор: Александр Крученых Автор: Александр Крученых'

finded = find_poem(request)

for d in finded:
	print(d['poem'])
	print({k: v for k, v in d.items() if k != 'poem'})
	print('------')

0it [00:00, ?it/s]


ValueError: could not convert string to float: '[-9.15968884e-03  2.05528717e-02 -2.54428294e-02 -1.48531245e-02\n  1.30124502e-02 -3.25524658e-02 -3.77138704e-02 -5.00245281e-02\n  1.49826836e-02  7.48177990e-03 -3.34518515e-02 -4.47052754e-02\n -3.25530278e-03 -7.04992712e-02  1.06156664e-02  5.77230798e-03\n  2.64669140e-03  1.40114902e-02 -5.50880954e-02  7.76266353e-03\n  6.48032501e-02  5.35690039e-02  2.64814147e-03  5.06590568e-02\n -4.29608952e-03 -5.32400869e-02  1.05339931e-02 -5.57884425e-02\n  1.24225346e-02  2.88664009e-02  3.32597084e-02  2.52631959e-04\n -2.37165708e-02  3.26288044e-02 -6.40792679e-03 -2.75005819e-04\n -2.01159604e-02  3.81386466e-02 -1.46067766e-02  2.94542406e-02\n  6.95203319e-02  3.31788473e-02  3.93318869e-02 -1.40048638e-02\n  7.89594278e-03 -3.69230174e-02  2.93830074e-02 -1.56003321e-02\n -4.95327171e-03 -2.02414319e-02 -1.05666518e-02 -1.10026738e-02\n -3.33168246e-02  1.54909901e-02  1.16648832e-02  5.39983669e-03\n  5.35076074e-02 -4.41131592